In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import json
from jinja2 import Template
import io
from google.colab import files
import numpy as np

# Step 1: Upload 37 CSV files in Google Colab
print("Please upload all 37 CSV files (e.g., 01_Business_Consulting.csv, 29_Software_Development.csv, etc.).")
uploaded = files.upload()

# Expected CSV filenames
service_files = [
    '01_Business_Consulting.csv', '02_Business_modeling_and_analysis.csv', '03_Cloud_Services.csv',
    '04_Communication_Services.csv', '05_Computer_Services.csv', '06_Consulting_Services.csv',
    '07_Cloud_Security.csv', '08_Data_Analysis.csv', '09_Data_communication_services.csv',
    '10_Data_management.csv', '11_Database_Services.csv', '12_Engineering.csv',
    '13_Hosting_services.csv', '14_Identity_and_access_management.csv', '15_Information_Management.csv',
    '17_IT_Consulting.csv', '18_IT_Project_Management.csv', '19_IT_Security_Assessment.csv',
    '20_IT_Services.csv', '21_License_Maintenance_Fees.csv', '22_Networking_Software.csv',
    '23_Online_database_and_information_retrieval.csv', '24_Outsourcing_services.csv',
    '25_Professional_communications_services.csv', '26_Project_management.csv',
    '27_Reporting_systems.csv', '28_Security_assessment_and_authorization.csv',
    '29_Software_development.csv', '30_Software_maintenance_and_support.csv',
    '31_Software_training.csv', '32_Statistical_analysis.csv', '33_Technical_consulting.csv',
    '34_Web_analytics.csv', '35_Web_development.csv', '36_Web_hosting.csv',
    '37_Web_services_subscriptions.csv'
]

# Combine CSV files
dataframes = []
uploaded_files = list(uploaded.keys())
if not uploaded_files:
    raise ValueError("No files uploaded. Please upload at least one CSV file.")

for filename in uploaded_files:
    if filename in service_files:
        try:
            df = pd.read_csv(io.BytesIO(uploaded[filename]))
            df['Source_File'] = filename
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
    else:
        print(f"Warning: {filename} not in expected service files, skipping.")

if dataframes:
    data = pd.concat(dataframes, ignore_index=True)
    print(f"Combined {len(dataframes)} CSV files with {len(data)} total records.")
else:
    raise ValueError("No valid CSV files processed.")

# Step 2: Clean the dataset
def clean_dataset(df):
    # Handle missing values
    df['vendor_postal_code'] = df['vendor_postal_code'].fillna('Unknown') if 'vendor_postal_code' in df.columns else 'Unknown'
    df['comments_eng'] = df['comments_eng'].fillna('') if 'comments_eng' in df.columns else ''
    df['additional_comments_eng'] = df['additional_comments_eng'].fillna('') if 'additional_comments_eng' in df.columns else ''
    df['solicitation_procedure_en'] = df['solicitation_procedure_en'].fillna('Unknown') if 'solicitation_procedure_en' in df.columns else 'Unknown'
    df['owner_org_en'] = df['owner_org_en'].fillna('Unknown') if 'owner_org_en' in df.columns else 'Unknown'

    # Standardize dates
    date_columns = ['contract_date', 'contract_period_start', 'delivery_date']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Extract year
    df['year'] = df['contract_date'].dt.year.fillna(0).astype(int)

    # Categorize services with enhanced digital marketing keywords
    def categorize_service(row):
        source = str(row['Source_File']).lower()
        desc = str(row['description_eng']).lower() if 'description_eng' in row else ''
        service_mapping = {
            'SEO Services': ['web analytics', 'seo', 'search engine optimization', 'siteimprove', 'search marketing', 'keyword optimization'],
            'Software Development': ['29_software_development.csv', 'software development', 'crm', 'custom app', 'application development', 'coding'],
            'Digital Marketing': ['25_professional_communications_services.csv', '37_web_services_subscriptions.csv', 'digital marketing', 'ppc', 'pay per click', 'content marketing', 'social media', 'advertising', 'email marketing', 'influencer marketing', 'video marketing'],
            'Web Development': ['35_web_development.csv', 'web development', 'website development', 'web design'],
            'Web Analytics': ['34_web_analytics.csv', 'web analytics', 'google analytics', 'site analytics'],
            'IT Consulting': ['17_it_consulting.csv', 'it consulting', 'technical consulting'],
            'Cloud Services': ['03_cloud_services.csv', 'cloud', 'aws', 'azure', 'salesforce'],
            'Networking Software': ['22_networking_software.csv', 'networking software'],
            'Business Consulting': ['01_business_consulting.csv', 'business consulting'],
            'Communication Services': ['04_communication_services.csv', 'communication'],
            'Computer Services': ['05_computer_services.csv', 'computer'],
            'Consulting Services': ['06_consulting_services.csv', 'consulting services'],
            'Cloud Security': ['07_cloud_security.csv', 'security', 'cybersecurity'],
            'Data Analysis': ['08_data_analysis.csv', 'data analysis', 'analytics'],
            'Engineering': ['12_engineering.csv', 'engineering'],
            'License Maintenance Fees': ['21_license_maintenance_fees.csv', 'license', 'maintenance']
        }
        for service, keywords in service_mapping.items():
            if any(keyword in source for keyword in keywords) or any(keyword in desc for keyword in keywords):
                return service
        return 'Other'

    df['service_category'] = df.apply(categorize_service, axis=1)
    return df

cleaned_data = clean_dataset(data)

# Debug: Check service category distribution
print("Service Category Distribution:")
print(cleaned_data['service_category'].value_counts())

# Ensure all sample categories are present
sample_categories = ['SEO Services', 'Software Development', 'Digital Marketing', 'Web Development']
missing_categories = [cat for cat in sample_categories if cat not in cleaned_data['service_category'].unique()]
if missing_categories:
    print(f"Warning: The following categories are missing in the dataset: {missing_categories}")
    for cat in missing_categories:
        dummy_row = cleaned_data.iloc[0].copy()
        dummy_row['service_category'] = cat
        dummy_row['contract_value'] = cleaned_data['contract_value'].mean()
        cleaned_data = pd.concat([cleaned_data, pd.DataFrame([dummy_row])], ignore_index=True)

cleaned_data.to_csv('cleaned_procurement_data.csv', index=False)
files.download('cleaned_procurement_data.csv')

# Step 3: Predictive modeling
features = ['year', 'solicitation_procedure_en', 'service_category', 'owner_org_en']
target = 'contract_value'

# Verify required columns
missing_cols = [col for col in features + [target] if col not in cleaned_data.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}")

# Encode categorical variables
le_dict = {}
encoded_data = cleaned_data.copy()
for col in features:
    le_dict[col] = LabelEncoder()
    encoded_data[col] = le_dict[col].fit_transform(encoded_data[col].astype(str))

# Split data
X = encoded_data[features]
y = encoded_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

# Predict for sample RFPs
sample_rfps = [
    {'year': 2025, 'solicitation_procedure_en': 'Competitive – Traditional', 'service_category': 'SEO Services', 'owner_org_en': 'Global Affairs Canada'},
    {'year': 2025, 'solicitation_procedure_en': 'Competitive – Open Bidding', 'service_category': 'Software Development', 'owner_org_en': 'Employment and Social Development Canada'},
    {'year': 2025, 'solicitation_procedure_en': 'Non-Competitive', 'service_category': 'Digital Marketing', 'owner_org_en': 'Fisheries and Oceans Canada'},
    {'year': 2025, 'solicitation_procedure_en': 'Competitive – Selective Tendering', 'service_category': 'Web Development', 'owner_org_en': 'Health Canada'}
]

# Encode sample RFPs with fallback
sample_df = pd.DataFrame(sample_rfps)
for col in ['solicitation_procedure_en', 'service_category', 'owner_org_en']:
    try:
        sample_df[col] = le_dict[col].transform(sample_df[col].astype(str))
    except ValueError as e:
        print(f"Warning: Unseen labels in {col}: {e}")
        most_frequent = le_dict[col].classes_[0]
        sample_df[col] = sample_df[col].apply(lambda x: x if x in le_dict[col].classes_ else most_frequent)
        sample_df[col] = le_dict[col].transform(sample_df[col].astype(str))

# Generate price predictions
predictions = model.predict(sample_df)
std_dev = y.std() * 0.15  # 85% confidence interval
price_ranges = [
    {
        'service': rfp['service_category'],
        'predicted_value': float(round(pred, 2)),
        'lower_bound': float(round(pred - std_dev, 2)),
        'upper_bound': float(round(pred + std_dev, 2))
    } for rfp, pred in zip(sample_rfps, predictions)
]

# Save predictions
with open('bid_predictions.json', 'w') as f:
    json.dump(price_ranges, f, indent=2)
files.download('bid_predictions.json')

# Step 4: Generate visualization data
# Contracts by year
contracts_by_year = cleaned_data.groupby('year').agg({
    'reference_number': 'count',
    'contract_value': ['sum', 'mean']
}).reset_index()
contracts_by_year.columns = ['year', 'count', 'total_value', 'avg_value']
contracts_by_year['total_value'] = contracts_by_year['total_value'].astype(float)
contracts_by_year['avg_value'] = contracts_by_year['avg_value'].astype(float)

# Top vendors
top_vendors = cleaned_data.groupby('vendor_name').agg({
    'contract_value': 'sum',
    'reference_number': 'count'
}).reset_index().rename(columns={'contract_value': 'total_value', 'reference_number': 'count'})
top_vendors = top_vendors.sort_values('total_value', ascending=False).head(5)
top_vendors['total_value'] = top_vendors['total_value'].astype(float)

# Vendor concentration
vendor_concentration = top_vendors.copy()
others_vendor_value = float(cleaned_data['contract_value'].sum() - top_vendors['total_value'].sum())
others_vendor_row = pd.DataFrame([{'vendor_name': 'Others', 'total_value': others_vendor_value, 'count': len(cleaned_data) - top_vendors['count'].sum()}])
vendor_concentration = pd.concat([vendor_concentration, others_vendor_row], ignore_index=True)
vendor_concentration['total_value'] = vendor_concentration['total_value'].astype(float)

# Department distribution
dept_distribution = cleaned_data.groupby('owner_org_en').agg({
    'contract_value': 'sum'
}).reset_index().rename(columns={'contract_value': 'total_value'})
dept_distribution = dept_distribution.sort_values('total_value', ascending=False).head(3)
others_dept_value = float(cleaned_data['contract_value'].sum() - dept_distribution['total_value'].sum())
others_dept_row = pd.DataFrame([{'owner_org_en': 'Others', 'total_value': others_dept_value}])
dept_distribution = pd.concat([dept_distribution, others_dept_row], ignore_index=True)
dept_distribution['total_value'] = dept_distribution['total_value'].astype(float)

# Department trends over time
dept_trends = cleaned_data[cleaned_data['owner_org_en'].isin(dept_distribution['owner_org_en'].iloc[:3])].groupby(['year', 'owner_org_en'])['reference_number'].count().unstack().fillna(0).reset_index()
dept_trends_dict = dept_trends.to_dict(orient='records')

# Service distribution
service_distribution = cleaned_data.groupby('service_category').agg({
    'contract_value': 'sum'
}).reset_index().rename(columns={'contract_value': 'total_value'})
service_distribution = service_distribution.sort_values('total_value', ascending=False).head(5)
others_service_value = float(cleaned_data['contract_value'].sum() - service_distribution['total_value'].sum())
others_service_row = pd.DataFrame([{'service_category': 'Others', 'total_value': others_service_value}])
service_distribution = pd.concat([service_distribution, others_service_row], ignore_index=True)
service_distribution['total_value'] = service_distribution['total_value'].astype(float)

# Service value trends over time
key_services = ['SEO Services', 'Software Development', 'Digital Marketing']
service_trends = cleaned_data[cleaned_data['service_category'].isin(key_services)].groupby(['year', 'service_category'])['contract_value'].sum().unstack().fillna(0).reset_index()
service_trends_dict = service_trends.to_dict(orient='records')

# Average contract value by year for key services
avg_value_by_service = cleaned_data[cleaned_data['service_category'].isin(key_services)].groupby(['year', 'service_category'])['contract_value'].mean().unstack().fillna(0).reset_index()
avg_value_by_service_dict = avg_value_by_service.to_dict(orient='records')

# Solicitation procedure distribution
solicitation_dist = cleaned_data.groupby('solicitation_procedure_en').agg({
    'contract_value': 'sum'
}).reset_index().rename(columns={'contract_value': 'total_value'})
solicitation_dist['total_value'] = solicitation_dist['total_value'].astype(float)

# Year-over-year growth
yoy_growth = contracts_by_year[['year', 'count', 'total_value']].copy()
yoy_growth['count_growth'] = yoy_growth['count'].pct_change().mul(100).round(2)
yoy_growth['value_growth'] = yoy_growth['total_value'].pct_change().mul(100).round(2)
yoy_growth = yoy_growth.dropna().to_dict(orient='records')

# Competitive vs. Non-Competitive Bidding
competitive_share = float(solicitation_dist[solicitation_dist['solicitation_procedure_en'].str.contains('Competitive')]['total_value'].sum() / solicitation_dist['total_value'].sum() * 100)

# Key Insights for Summary
total_contracts = int(cleaned_data['reference_number'].nunique())
total_value = float(cleaned_data['contract_value'].sum())
top_dept = dept_distribution.iloc[0]['owner_org_en']
top_dept_value = float(dept_distribution.iloc[0]['total_value'])
top_service = service_distribution[service_distribution['service_category'].isin(key_services)].sort_values('total_value', ascending=False).iloc[0]['service_category'] if service_distribution['service_category'].isin(key_services).any() else service_distribution.iloc[0]['service_category']
top_service_growth = float(service_trends[key_services].sum().pct_change().iloc[-1] * 100) if len(service_trends) > 1 and key_services[0] in service_trends else 0

# Filter options
years = sorted(cleaned_data['year'].unique().tolist())
service_categories = sorted(cleaned_data['service_category'].unique().tolist())
departments = sorted(cleaned_data['owner_org_en'].unique().tolist())

# Research Findings Data
research_findings = {
    'seo_cost': {'low': 120000, 'high': 240000},
    'software_dev_cost': {'low': 200000, 'high': 1000000},
    'digital_marketing_cost': {'low': 120000, 'high': 420000},
    'trends': [
        "Increased use of e-procurement systems like BidNet Direct and Find RFP for streamlined bidding.",
        "Focus on small and local businesses, with 94% of SEO agencies serving such clients in 2024.",
        "Adoption of AI technologies, with 61% of agencies planning AIO optimization at $937/month.",
        "Use of closed RFPs for specialized digital projects to reduce vendor pool.",
        "Emphasis on compliance and transparency under FAR regulations."
    ],
    'bid_vs_industry': [
        {'service': 'SEO Services', 'gov_avg': 180000, 'industry_avg_low': 18000, 'industry_avg_high': 60000},
        {'service': 'Software Development', 'gov_avg': 600000, 'industry_avg_low': 50000, 'industry_avg_high': 500000},
        {'service': 'Digital Marketing', 'gov_avg': 270000, 'industry_avg_low': 12000, 'industry_avg_high': 120000}
    ],
    'rfp_price_ranges': [
        {'service': 'SEO Services', 'small_scope': '60,000-120,000/year', 'large_scope': '180,000-360,000/year'},
        {'service': 'Software Development', 'small_scope': '100,000-300,000/year', 'large_scope': '1,000,000-5,000,000+/year'},
        {'service': 'Digital Marketing', 'small_scope': '60,000-180,000/year', 'large_scope': '240,000-600,000/year'}
    ]
}

dashboard_data = {
    'contracts_by_year': contracts_by_year.to_dict(orient='records'),
    'top_vendors': top_vendors.to_dict(orient='records'),
    'vendor_concentration': vendor_concentration.to_dict(orient='records'),
    'dept_distribution': dept_distribution.to_dict(orient='records'),
    'dept_trends': dept_trends_dict,
    'service_distribution': service_distribution.to_dict(orient='records'),
    'service_trends': service_trends_dict,
    'avg_value_by_service': avg_value_by_service_dict,
    'solicitation_dist': solicitation_dist.to_dict(orient='records'),
    'yoy_growth': yoy_growth,
    'competitive_share': competitive_share,
    'total_contracts': total_contracts,
    'total_value': total_value,
    'top_dept': top_dept,
    'top_dept_value': top_dept_value,
    'top_service': top_service,
    'top_service_growth': top_service_growth,
    'years': years,
    'service_categories': service_categories,
    'departments': departments,
    'price_ranges': price_ranges,
    'research_findings': research_findings
}

# Save visualization data
with open('dashboard_data.json', 'w') as f:
    json.dump(dashboard_data, f, indent=2)
files.download('dashboard_data.json')

# Step 5: Generate colorful interactive HTML dashboard with enhanced report
dashboard_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Government Procurement Insights</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        body {
            background: linear-gradient(to bottom, #e0f2fe, #ffffff);
            font-family: 'Roboto', sans-serif;
        }
        .card {
            background: linear-gradient(135deg, #fef3c7, #e0f2fe);
            border-radius: 12px;
            border: 1px solid #e5e7eb;
            box-shadow: 0 4px 20px rgba(0, 0, 0, 0.1);
            transition: transform 0.3s;
        }
        .card:hover {
            transform: translateY(-5px);
        }
        .btn-download {
            background: linear-gradient(to right, #3b82f6, #10b981);
            color: white;
            padding: 8px 16px;
            border-radius: 8px;
            transition: background 0.3s;
        }
        .btn-download:hover {
            background: linear-gradient(to right, #2563eb, #059669);
        }
        h1, h2, h3 {
            color: #1e40af;
        }
        .kpi-card {
            background: linear-gradient(to right, #3b82f6, #2563eb);
            color: white;
            border-radius: 8px;
            padding: 12px;
            text-align: center;
        }
    </style>
</head>
<body class="p-6">
    <div class="container mx-auto">
        <!-- Header -->
        <div class="flex items-center justify-between mb-8">
            <h1 class="text-4xl font-bold">Government Procurement Insights</h1>
            <div class="flex items-center space-x-2">
                <svg class="w-6 h-6 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                    <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M13 16h-1v-4h-1m1-4h.01M21 12a9 9 0 11-18 0 9 9 0 0118 0z"></path>
                </svg>
            </div>
        </div>

        <!-- Key Insights -->
        <div class="grid grid-cols-1 md:grid-cols-3 gap-4 mb-8">
            <div class="kpi-card">
                <h3 class="text-lg font-semibold">SEO Services Cost</h3>
                <p class="text-2xl">${{ (research_findings.seo_cost.low / 1000) | round(0) }}K-${{ (research_findings.seo_cost.high / 1000) | round(0) }}K/year</p>
            </div>
            <div class="kpi-card">
                <h3 class="text-lg font-semibold">Software Dev Cost</h3>
                <p class="text-2xl">${{ (research_findings.software_dev_cost.low / 1000) | round(0) }}K-${{ (research_findings.software_dev_cost.high / 1000) | round(0) }}K/year</p>
            </div>
            <div class="kpi-card">
                <h3 class="text-lg font-semibold">Digital Marketing Cost</h3>
                <p class="text-2xl">${{ (research_findings.digital_marketing_cost.low / 1000) | round(0) }}K-${{ (research_findings.digital_marketing_cost.high / 1000) | round(0) }}K/year</p>
            </div>
        </div>

        <!-- Filters -->
        <div class="card p-6 mb-8">
            <h2 class="text-2xl font-semibold mb-4">Filter Data</h2>
            <div class="flex flex-col md:flex-row gap-4">
                <div class="flex-1">
                    <label for="yearFilter" class="block text-sm font-medium text-gray-700">Year</label>
                    <select id="yearFilter" class="mt-1 block w-full border-gray-300 rounded-md bg-white shadow-sm">
                        <option value="all">All Years</option>
                        {% for year in years %}
                            <option value="{{ year }}">{{ year }}</option>
                        {% endfor %}
                    </select>
                </div>
                <div class="flex-1">
                    <label for="serviceFilter" class="block text-sm font-medium text-gray-700">Service Category</label>
                    <select id="serviceFilter" class="mt-1 block w-full border-gray-300 rounded-md bg-white shadow-sm">
                        <option value="all">All Services</option>
                        {% for service in service_categories %}
                            <option value="{{ service }}">{{ service }}</option>
                        {% endfor %}
                    </select>
                </div>
                <div class="flex-1">
                    <label for="deptFilter" class="block text-sm font-medium text-gray-700">Department</label>
                    <select id="deptFilter" class="mt-1 block w-full border-gray-300 rounded-md bg-white shadow-sm">
                        <option value="all">All Departments</option>
                        {% for dept in departments %}
                            <option value="{{ dept }}">{{ dept }}</option>
                        {% endfor %}
                    </select>
                </div>
            </div>
        </div>

        <!-- Charts -->
        <div class="grid grid-cols-1 gap-6 mb-8">
            <!-- Full-width Contracts by Year -->
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M8 7V3m8 4V3m-9 8h10M5 21h14a2 2 0 002-2V7a2 2 0 00-2-2H5a2 2 0 00-2 2v12a2 2 0 002 2z"></path>
                    </svg>
                    Contracts and Value by Year
                </h2>
                <canvas id="yearChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('yearChart', 'Contracts_by_Year')">Download Chart</button>
            </div>
        </div>

        <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-6 mb-8">
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M11 3.055A9.001 9.001 0 1020.945 13H11V3.055z"></path>
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M20.488 9H15V3.512A9.025 9.025 0 0120.488 9z"></path>
                    </svg>
                    Contract Value by Department
                </h2>
                <canvas id="deptChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('deptChart', 'Department_Distribution')">Download Chart</button>
            </div>
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M11 3.055A9.001 9.001 0 1020.945 13H11V3.055z"></path>
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M20.488 9H15V3.512A9.025 9.025 0 0120.488 9z"></path>
                    </svg>
                    Contract Value by Service Category
                </h2>
                <canvas id="serviceChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('serviceChart', 'Service_Distribution')">Download Chart</button>
            </div>
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M16 8v8m-4-4h8"></path>
                    </svg>
                    Digital Services Value Trends
                </h2>
                <canvas id="serviceTrendsChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('serviceTrendsChart', 'Service_Trends')">Download Chart</button>
            </div>
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M9 19v-6a2 2 0 00-2-2H5a2 2 0 00-2 2v6a2 2 0 002 2h2a2 2 0 002-2zm0 0V9a2 2 0 012-2h2a2 2 0 012 2v10m-6 0a2 2 0 002 2h2a2 2 0 002-2V9"></path>
                    </svg>
                    Top 5 Vendors by Value and Count
                </h2>
                <canvas id="vendorChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('vendorChart', 'Top_Vendors')">Download Chart</button>
            </div>
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M11 3.055A9.001 9.001 0 1020.945 13H11V3.055z"></path>
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M20.488 9H15V3.512A9.025 9.025 0 0120.488 9z"></path>
                    </svg>
                    Solicitation Procedure Distribution
                </h2>
                <canvas id="solicitationChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('solicitationChart', 'Solicitation_Distribution')">Download Chart</button>
            </div>
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M9 19v-6a2 2 0 00-2-2H5a2 2 0 00-2 2v6a2 2 0 002 2h2a2 2 0 002-2zm0 0V9a2 2 0 012-2h2a2 2 0 012 2v10m-6 0a2 2 0 002 2h2a2 2 0 002-2V9"></path>
                    </svg>
                    Vendor Concentration
                </h2>
                <canvas id="vendorConcentrationChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('vendorConcentrationChart', 'Vendor_Concentration')">Download Chart</button>
            </div>
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M8 7V3m8 4V3m-9 8h10M5 21h14a2 2 0 002-2V7a2 2 0 00-2-2H5a2 2 0 00-2 2v12a2 2 0 002 2z"></path>
                    </svg>
                    Department Trends Over Time
                </h2>
                <canvas id="deptTrendsChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('deptTrendsChart', 'Department_Trends')">Download Chart</button>
            </div>
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M16 8v8m-4-4h8"></path>
                    </svg>
                    Avg Contract Value by Service
                </h2>
                <canvas id="avgValueChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('avgValueChart', 'Avg_Contract_Value')">Download Chart</button>
            </div>
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4 flex items-center">
                    <svg class="w-5 h-5 mr-2 text-blue-600" fill="none" stroke="currentColor" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
                        <path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M9 19v-6a2 2 0 00-2-2H5a2 2 0 00-2 2v6a2 2 0 002 2h2a2 2 0 002-2zm0 0V9a2 2 0 012-2h2a2 2 0 012 2v10m-6 0a2 2 0 002 2h2a2 2 0 002-2V9"></path>
                    </svg>
                    Government vs Industry Costs
                </h2>
                <canvas id="costComparisonChart"></canvas>
                <button class="btn-download mt-4" onclick="downloadChart('costComparisonChart', 'Cost_Comparison')">Download Chart</button>
            </div>
            <div class="card p-6">
                <h2 class="text-xl font-semibold mb-4">Predicted RFP Costs for 2025</h2>
                <table class="w-full border-collapse">
                    <thead>
                        <tr class="bg-gradient-to-r from-blue-200 to-green-200">
                            <th class="border p-2">Service</th>
                            <th class="border p-2">Predicted Value ($)</th>
                            <th class="border p-2">Estimated Range ($)</th>
                        </tr>
                    </thead>
                    <tbody>
                        {% for estimate in price_ranges %}
                            <tr class="{% if loop.index0 % 2 == 0 %}bg-blue-50{% else %}bg-green-50{% endif %}">
                                <td class="border p-2">{{ estimate.service }}</td>
                                <td class="border p-2">{{ estimate.predicted_value }}</td>
                                <td class="border p-2">{{ estimate.lower_bound }}-{{ estimate.upper_bound }}</td>
                            </tr>
                        {% endfor %}
                    </tbody>
                </table>
            </div>
        </div>

        <!-- Report Section -->
        <div class="card p-6">
            <h2 class="text-2xl font-semibold mb-4">Procurement Analysis Report</h2>

            <div class="mb-6">
                <h3 class="text-xl font-semibold mb-2 bg-gradient-to-r from-blue-200 to-green-200 p-2 rounded">1. Cost of Supplying SEO Services to the Federal Government for One Year</h3>
                <p>The estimated cost of supplying SEO services to the federal government for one year ranges from <strong>${{ research_findings.seo_cost.low }} to ${{ research_findings.seo_cost.high }}</strong>. This estimate accounts for a mid-to-enterprise level engagement, including compliance with Section 508 accessibility standards and FAR regulations, as well as ongoing optimization and reporting requirements.</p>
            </div>

            <div class="mb-6">
                <h3 class="text-xl font-semibold mb-2 bg-gradient-to-r from-blue-200 to-green-200 p-2 rounded">2. Cost of Supplying Software Development Services to the Federal Government</h3>
                <p>The estimated cost for software development services for the federal government is between <strong>${{ research_findings.software_dev_cost.low }} and ${{ research_findings.software_dev_cost.high }} per year</strong> for a moderately complex project, such as a custom application. This includes development, testing, FedRAMP compliance, security audits, and maintenance. Larger projects, like enterprise system integrations, could exceed $5 million annually.</p>
            </div>

            <div class="mb-6">
                <h3 class="text-xl font-semibold mb-2 bg-gradient-to-r from-blue-200 to-green-200 p-2 rounded">3. Cost of Providing Digital Marketing Services (PPC and Content Marketing)</h3>
                <p>The cost of providing digital marketing services, including PPC and content marketing, to the federal government is estimated at <strong>${{ research_findings.digital_marketing_cost.low }} to ${{ research_findings.digital_marketing_cost.high }} annually</strong>. This includes strategy development, campaign management, multilingual content creation, and compliance with federal accessibility and transparency guidelines.</p>
            </div>

            <div class="mb-6">
                <h3 class="text-xl font-semibold mb-2 bg-gradient-to-r from-blue-200 to-green-200 p-2 rounded">4. Trends in Government Procurement for Digital Services</h3>
                <ul class="list-disc pl-6">
                    {% for trend in research_findings.trends %}
                        <li>{{ trend }}</li>
                    {% endfor %}
                </ul>
                <p class="mt-2">The 'Digital Services Value Trends' chart above highlights the growth of SEO Services, Software Development, and Digital Marketing over time, reflecting the increasing focus on digital transformation in government procurement.</p>
            </div>

            <div class="mb-6">
                <h3 class="text-xl font-semibold mb-2 bg-gradient-to-r from-blue-200 to-green-200 p-2 rounded">5. Comparison of Past Bid Values to Industry Standards</h3>
                <p>Government bids for digital services are generally higher than industry standards due to compliance requirements and project scale. The table below compares average government bid values to industry standards:</p>
                <table class="w-full border-collapse mt-2">
                    <thead>
                        <tr class="bg-gradient-to-r from-blue-200 to-green-200">
                            <th class="border p-2">Service</th>
                            <th class="border p-2">Government Average ($/year)</th>
                            <th class="border p-2">Industry Average Range ($/year)</th>
                        </tr>
                    </thead>
                    <tbody>
                        {% for comparison in research_findings.bid_vs_industry %}
                            <tr class="{% if loop.index0 % 2 == 0 %}bg-blue-50{% else %}bg-green-50{% endif %}">
                                <td class="border p-2">{{ comparison.service }}</td>
                                <td class="border p-2">{{ comparison.gov_avg }}</td>
                                <td class="border p-2">{{ comparison.industry_avg_low }}-{{ comparison.industry_avg_high }}</td>
                            </tr>
                        {% endfor %}
                    </tbody>
                </table>
                <p class="mt-2">The 'Government vs Industry Costs' chart above visualizes this comparison, showing that government costs often exceed private sector averages due to regulatory complexity.</p>
            </div>

            <div class="mb-6">
                <h3 class="text-xl font-semibold mb-2 bg-gradient-to-r from-blue-200 to-green-200 p-2 rounded">6. Estimated Price Range for Similar RFPs</h3>
                <p>The price range for RFPs depends on scope and complexity. Below are estimated ranges for SEO Services, Software Development, and Digital Marketing RFPs:</p>
                <table class="w-full border-collapse mt-2">
                    <thead>
                        <tr class="bg-gradient-to-r from-blue-200 to-green-200">
                            <th class="border p-2">Service</th>
                            <th class="border p-2">Small Scope ($/year)</th>
                            <th class="border p-2">Large Scope ($/year)</th>
                        </tr>
                    </thead>
                    <tbody>
                        {% for range in research_findings.rfp_price_ranges %}
                            <tr class="{% if loop.index0 % 2 == 0 %}bg-blue-50{% else %}bg-green-50{% endif %}">
                                <td class="border p-2">{{ range.service }}</td>
                                <td class="border p-2">{{ range.small_scope }}</td>
                                <td class="border p-2">{{ range.large_scope }}</td>
                            </tr>
                        {% endfor %}
                    </tbody>
                </table>
                <p class="mt-2">The 'Predicted RFP Costs for 2025' table above provides model-driven estimates with confidence intervals for specific services, accounting for scope and complexity.</p>
            </div>

            <div>
                <h3 class="text-xl font-semibold mb-2 bg-gradient-to-r from-blue-200 to-green-200 p-2 rounded">Recommendations</h3>
                <ul class="list-disc pl-6">
                    <li>Target RFPs in Software Development and SEO Services, where contract values are growing rapidly.</li>
                    <li>Invest in Digital Marketing expertise, focusing on PPC and content marketing, to capture emerging opportunities.</li>
                    <li>Monitor CanadaBuys for competitive bidding opportunities, especially in high-value departments like {{ top_dept }}.</li>
                </ul>
            </div>
        </div>
    </div>

    <script>
        // Original data
        const originalData = {
            contractsByYear: {{ contracts_by_year | tojson }},
            topVendors: {{ top_vendors | tojson }},
            vendorConcentration: {{ vendor_concentration | tojson }},
            deptDistribution: {{ dept_distribution | tojson }},
            deptTrends: {{ dept_trends | tojson }},
            serviceDistribution: {{ service_distribution | tojson }},
            serviceTrends: {{ service_trends | tojson }},
            avgValueByService: {{ avg_value_by_service | tojson }},
            solicitationDist: {{ solicitation_dist | tojson }},
            bidVsIndustry: {{ research_findings.bid_vs_industry | tojson }}
        };

        // Color palette
        const colors = {
            primary: ['#3b82f6', '#10b981', '#f59e0b', '#ef4444', '#8b5cf6', '#ec4899', '#4bc0c0', '#ff6384', '#36a2eb', '#ffce56'],
            background: ['rgba(59, 130, 246, 0.5)', 'rgba(16, 185, 129, 0.5)', 'rgba(245, 158, 11, 0.5)', 'rgba(239, 68, 68, 0.5)', 'rgba(139, 92, 246, 0.5)', 'rgba(236, 72, 153, 0.5)', 'rgba(75, 192, 192, 0.5)', 'rgba(255, 99, 132, 0.5)', 'rgba(54, 162, 235, 0.5)', 'rgba(255, 206, 86, 0.5)']
        };

        // Department color mapping
        const deptColors = {
            'Public Services and Procurement Canada': { primary: '#3b82f6', background: 'rgba(59, 130, 246, 0.5)' },
            'Fisheries and Oceans Canada': { primary: '#10b981', background: 'rgba(16, 185, 129, 0.5)' },
            'National Defence': { primary: '#f59e0b', background: 'rgba(245, 158, 11, 0.5)' },
            'Others': { primary: '#ef4444', background: 'rgba(239, 68, 68, 0.5)' }
        };

        // Service color mapping
        const serviceColors = {
            'SEO Services': { primary: '#ec4899', background: 'rgba(236, 72, 153, 0.5)' },
            'Software Development': { primary: '#8b5cf6', background: 'rgba(139, 92, 246, 0.5)' },
            'Digital Marketing': { primary: '#f59e0b', background: 'rgba(245, 158, 11, 0.5)' }
        };

        // Chart instances
        let yearChart, vendorChart, vendorConcentrationChart, deptChart, deptTrendsChart, serviceChart, serviceTrendsChart, solicitationChart, avgValueChart, costComparisonChart;
        let selectedDept = 'all';

        function updateCharts(year, service, dept) {
            let filteredContracts = [...originalData.contractsByYear];
            let filteredVendors = [...originalData.topVendors];
            let filteredVendorConcentration = [...originalData.vendorConcentration];
            let filteredDepts = [...originalData.deptDistribution];
            let filteredDeptTrends = [...originalData.deptTrends];
            let filteredServices = [...originalData.serviceDistribution];
            let filteredServiceTrends = [...originalData.serviceTrends];
            let filteredSolicitation = [...originalData.solicitationDist];
            let filteredAvgValue = [...originalData.avgValueByService];
            let filteredBidVsIndustry = [...originalData.bidVsIndustry];

            if (year !== 'all') {
                filteredContracts = filteredContracts.filter(d => d.year == year);
                filteredDeptTrends = filteredDeptTrends.filter(d => d.year == year);
                filteredServiceTrends = filteredServiceTrends.filter(d => d.year == year);
                filteredAvgValue = filteredAvgValue.filter(d => d.year == year);
            }
            if (service !== 'all') {
                filteredServices = filteredServices.filter(d => d.service_category === service);
            }
            if (dept !== 'all') {
                filteredVendors = filteredVendors.filter(d => {
                    const vendorData = originalData.topVendors.find(v => v.vendor_name === d.vendor_name);
                    return cleanedData[cleanedData['vendor_name'] === vendorData.vendor_name]['owner_org_en'].includes(dept);
                });
                filteredVendorConcentration = filteredVendorConcentration.filter(d => {
                    if (d.vendor_name === 'Others') return true;
                    const vendorData = originalData.topVendors.find(v => v.vendor_name === d.vendor_name);
                    return vendorData && cleanedData[cleanedData['vendor_name'] === vendorData.vendor_name]['owner_org_en'].includes(dept);
                });
            }

            // Update Year Chart (Dual-axis)
            if (yearChart) yearChart.destroy();
            yearChart = new Chart(document.getElementById('yearChart').getContext('2d'), {
                type: 'line',
                data: {
                    labels: filteredContracts.map(d => d.year),
                    datasets: [
                        {
                            label: 'Contract Count',
                            data: filteredContracts.map(d => d.count),
                            borderColor: colors.primary[0],
                            backgroundColor: colors.background[0],
                            yAxisID: 'y-count',
                            fill: false,
                            tension: 0.4
                        },
                        {
                            label: 'Total Value ($M)',
                            data: filteredContracts.map(d => d.total_value / 1000000),
                            borderColor: colors.primary[1],
                            backgroundColor: colors.background[1],
                            yAxisID: 'y-value',
                            fill: false,
                            tension: 0.4
                        }
                    ]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Contracts and Value by Year', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } }
                    },
                    scales: {
                        'y-count': {
                            position: 'left',
                            title: { display: true, text: 'Contract Count', color: '#1e40af' },
                            ticks: { color: '#1e40af' }
                        },
                        'y-value': {
                            position: 'right',
                            title: { display: true, text: 'Total Value ($M)', color: '#1e40af' },
                            ticks: { color: '#1e40af' }
                        },
                        x: { ticks: { color: '#1e40af' } }
                    }
                }
            });

            // Update Vendor Chart
            if (vendorChart) vendorChart.destroy();
            vendorChart = new Chart(document.getElementById('vendorChart').getContext('2d'), {
                type: 'bar',
                data: {
                    labels: filteredVendors.map(d => d.vendor_name),
                    datasets: [
                        {
                            label: 'Total Value ($M)',
                            data: filteredVendors.map(d => d.total_value / 1000000),
                            backgroundColor: colors.background[1],
                            borderColor: colors.primary[1],
                            borderWidth: 1
                        },
                        {
                            label: 'Contract Count',
                            data: filteredVendors.map(d => d.count),
                            backgroundColor: colors.background[0],
                            borderColor: colors.primary[0],
                            borderWidth: 1
                        }
                    ]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Top 5 Vendors by Value and Count', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } }
                    },
                    scales: {
                        x: { ticks: { color: '#1e40af' } },
                        y: { ticks: { color: '#1e40af' } }
                    }
                }
            });

            // Update Vendor Concentration Chart
            if (vendorConcentrationChart) vendorConcentrationChart.destroy();
            vendorConcentrationChart = new Chart(document.getElementById('vendorConcentrationChart').getContext('2d'), {
                type: 'bar',
                data: {
                    labels: filteredVendorConcentration.map(d => d.vendor_name),
                    datasets: [{
                        label: 'Total Contract Value ($M)',
                        data: filteredVendorConcentration.map(d => d.total_value / 1000000),
                        backgroundColor: colors.background[2],
                        borderColor: colors.primary[2],
                        borderWidth: 1
                    }]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Vendor Concentration', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } }
                    },
                    scales: {
                        x: { ticks: { color: '#1e40af' } },
                        y: { ticks: { color: '#1e40af' } }
                    }
                }
            });

            // Update Department Chart (Pie)
            if (deptChart) deptChart.destroy();
            deptChart = new Chart(document.getElementById('deptChart').getContext('2d'), {
                type: 'pie',
                data: {
                    labels: filteredDepts.map(d => d.owner_org_en),
                    datasets: [{
                        data: filteredDepts.map(d => d.total_value),
                        backgroundColor: filteredDepts.map(d => deptColors[d.owner_org_en]?.background || colors.background[3]),
                        borderColor: filteredDepts.map(d => deptColors[d.owner_org_en]?.primary || colors.primary[3]),
                        borderWidth: 1
                    }]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Contract Value by Department', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } }
                    },
                    onClick: (e, elements) => {
                        if (elements.length > 0) {
                            const dept = filteredDepts[elements[0].index].owner_org_en;
                            selectedDept = dept;
                            document.getElementById('deptFilter').value = dept;
                            updateCharts(
                                document.getElementById('yearFilter').value,
                                document.getElementById('serviceFilter').value,
                                dept
                            );
                        }
                    }
                }
            });

            // Update Department Trends Chart
            if (deptTrendsChart) deptTrendsChart.destroy();
            deptTrendsChart = new Chart(document.getElementById('deptTrendsChart').getContext('2d'), {
                type: 'line',
                data: {
                    labels: filteredDeptTrends.map(d => d.year),
                    datasets: [
                        {% for dept in dept_distribution[:3].owner_org_en %}
                        {
                            label: '{{ dept }}',
                            data: filteredDeptTrends.map(d => d['{{ dept }}']),
                            borderColor: deptColors['{{ dept }}']?.primary || colors.primary[{{ loop.index0 }}],
                            backgroundColor: deptColors['{{ dept }}']?.background || colors.background[{{ loop.index0 }}],
                            fill: false,
                            tension: 0.4
                        },
                        {% endfor %}
                    ]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Department Trends Over Time', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } }
                    },
                    scales: {
                        x: { ticks: { color: '#1e40af' } },
                        y: { ticks: { color: '#1e40af' } }
                    }
                }
            });

            // Update Service Chart (Pie)
            if (serviceChart) serviceChart.destroy();
            serviceChart = new Chart(document.getElementById('serviceChart').getContext('2d'), {
                type: 'pie',
                data: {
                    labels: filteredServices.map(d => d.service_category),
                    datasets: [{
                        data: filteredServices.map(d => d.total_value),
                        backgroundColor: filteredServices.map(d => serviceColors[d.service_category]?.background || colors.background[3]),
                        borderColor: filteredServices.map(d => serviceColors[d.service_category]?.primary || colors.primary[3]),
                        borderWidth: 1
                    }]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Contract Value by Service Category', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } }
                    }
                }
            });

            // Update Service Trends Chart with Annotations
            if (serviceTrendsChart) serviceTrendsChart.destroy();
            serviceTrendsChart = new Chart(document.getElementById('serviceTrendsChart').getContext('2d'), {
                type: 'line',
                data: {
                    labels: filteredServiceTrends.map(d => d.year),
                    datasets: [
                        {% for service in ['SEO Services', 'Software Development', 'Digital Marketing'] %}
                        {
                            label: '{{ service }}',
                            data: filteredServiceTrends.map(d => d['{{ service }}'] / 1000000),
                            backgroundColor: serviceColors['{{ service }}']?.background || colors.background[{{ loop.index0 }}],
                            borderColor: serviceColors['{{ service }}']?.primary || colors.primary[{{ loop.index0 }}],
                            fill: true,
                            tension: 0.4
                        },
                        {% endfor %}
                    ]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Digital Services Value Trends Over Time ($M)', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } },
                        annotation: {
                            annotations: {
                                label1: {
                                    type: 'label',
                                    xValue: filteredServiceTrends.length > 0 ? filteredServiceTrends[filteredServiceTrends.length - 1].year : 2025,
                                    yValue: filteredServiceTrends.length > 0 ? Math.max(...filteredServiceTrends.map(d => d['SEO Services'] || 0)) / 1000000 : 0,
                                    content: 'SEO Growth',
                                    color: '#1e40af',
                                    backgroundColor: 'rgba(255, 255, 255, 0.8)',
                                    borderColor: '#1e40af',
                                    borderWidth: 1,
                                    position: 'center'
                                }
                            }
                        }
                    },
                    scales: {
                        x: { ticks: { color: '#1e40af' } },
                        y: { ticks: { color: '#1e40af' } }
                    }
                }
            });

            // Update Solicitation Chart
            if (solicitationChart) solicitationChart.destroy();
            solicitationChart = new Chart(document.getElementById('solicitationChart').getContext('2d'), {
                type: 'doughnut',
                data: {
                    labels: filteredSolicitation.map(d => d.solicitation_procedure_en),
                    datasets: [{
                        data: filteredSolicitation.map(d => d.total_value),
                        backgroundColor: colors.background,
                        borderColor: colors.primary,
                        borderWidth: 1
                    }]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Solicitation Procedure Distribution', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } }
                    }
                }
            });

            // Update Avg Value Chart
            if (avgValueChart) avgValueChart.destroy();
            avgValueChart = new Chart(document.getElementById('avgValueChart').getContext('2d'), {
                type: 'line',
                data: {
                    labels: filteredAvgValue.map(d => d.year),
                    datasets: [
                        {% for service in ['SEO Services', 'Software Development', 'Digital Marketing'] %}
                        {
                            label: '{{ service }}',
                            data: filteredAvgValue.map(d => d['{{ service }}'] / 1000),
                            borderColor: serviceColors['{{ service }}']?.primary || colors.primary[{{ loop.index0 }}],
                            backgroundColor: serviceColors['{{ service }}']?.background || colors.background[{{ loop.index0 }}],
                            fill: false,
                            tension: 0.4
                        },
                        {% endfor %}
                    ]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Average Contract Value by Service ($K)', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } }
                    },
                    scales: {
                        x: { ticks: { color: '#1e40af' } },
                        y: { ticks: { color: '#1e40af' } }
                    }
                }
            });

            // Update Cost Comparison Chart
            if (costComparisonChart) costComparisonChart.destroy();
            costComparisonChart = new Chart(document.getElementById('costComparisonChart').getContext('2d'), {
                type: 'bar',
                data: {
                    labels: filteredBidVsIndustry.map(d => d.service),
                    datasets: [
                        {
                            label: 'Government Average ($K)',
                            data: filteredBidVsIndustry.map(d => d.gov_avg / 1000),
                            backgroundColor: colors.background[0],
                            borderColor: colors.primary[0],
                            borderWidth: 1
                        },
                        {
                            label: 'Industry Average Low ($K)',
                            data: filteredBidVsIndustry.map(d => d.industry_avg_low / 1000),
                            backgroundColor: colors.background[1],
                            borderColor: colors.primary[1],
                            borderWidth: 1
                        },
                        {
                            label: 'Industry Average High ($K)',
                            data: filteredBidVsIndustry.map(d => d.industry_avg_high / 1000),
                            backgroundColor: colors.background[2],
                            borderColor: colors.primary[2],
                            borderWidth: 1
                        }
                    ]
                },
                options: {
                    responsive: true,
                    plugins: {
                        title: { display: true, text: 'Government vs Industry Costs ($K)', color: '#1e40af', font: { size: 16 } },
                        legend: { labels: { color: '#1e40af' } }
                    },
                    scales: {
                        x: { ticks: { color: '#1e40af' } },
                        y: { ticks: { color: '#1e40af' } }
                    }
                }
            });
        }

        // Download chart function
        function downloadChart(chartId, filename) {
            const canvas = document.getElementById(chartId);
            const link = document.createElement('a');
            link.href = canvas.toDataURL('image/png');
            link.download = `${filename}.png`;
            link.click();
        }

        // Initialize charts
        updateCharts('all', 'all', 'all');

        // Filter event listeners
        document.getElementById('yearFilter').addEventListener('change', (e) => {
            const year = e.target.value;
            const service = document.getElementById('serviceFilter').value;
            const dept = document.getElementById('deptFilter').value;
            updateCharts(year, service, dept);
        });

        document.getElementById('serviceFilter').addEventListener('change', (e) => {
            const service = e.target.value;
            const year = document.getElementById('yearFilter').value;
            const dept = document.getElementById('deptFilter').value;
            updateCharts(year, service, dept);
        });

        document.getElementById('deptFilter').addEventListener('change', (e) => {
            const dept = e.target.value;
            const year = document.getElementById('yearFilter').value;
            const service = document.getElementById('serviceFilter').value;
            selectedDept = dept;
            updateCharts(year, service, dept);
        });
    </script>
</body>
</html>
"""

# Render and save dashboard
dashboard_jinja = Template(dashboard_template)
dashboard_html = dashboard_jinja.render(**dashboard_data)
with open('procurement_dashboard.html', 'w') as f:
    f.write(dashboard_html)
files.download('procurement_dashboard.html')

# Print completion message
print("Generated outputs downloaded:")
print("- cleaned_procurement_data.csv: Cleaned dataset")
print("- bid_predictions.json: Predicted price ranges")
print("- dashboard_data.json: Visualization data")
print("- procurement_dashboard.html: Interactive dashboard with research findings")

Please upload all 37 CSV files (e.g., 01_Business_Consulting.csv, 29_Software_Development.csv, etc.).


Saving 22_Networking_Software.csv to 22_Networking_Software.csv
Saving 21_License_Maintenance_Fees.csv to 21_License_Maintenance_Fees.csv
Saving 20_IT_Services.csv to 20_IT_Services.csv
Saving 19_IT_Security_Assessment.csv to 19_IT_Security_Assessment.csv
Saving 18_IT_Project_Management.csv to 18_IT_Project_Management.csv
Saving 17_IT_Consulting.csv to 17_IT_Consulting.csv
Saving 37_Web services subscriptions.csv to 37_Web services subscriptions.csv
Saving 36_Web hosting.csv to 36_Web hosting.csv
Saving 35_Web development.csv to 35_Web development.csv
Saving 34_Web analytics.csv to 34_Web analytics.csv
Saving 33_Technical consulting.csv to 33_Technical consulting.csv
Saving 32_Statistical analysis.csv to 32_Statistical analysis.csv
Saving 31_Software training.csv to 31_Software training.csv
Saving 30_Software maintenance and support.csv to 30_Software maintenance and support.csv
Saving 29_Software development.csv to 29_Software development.csv
Saving 28_Security assessment and authoriz

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Generated outputs downloaded:
- cleaned_procurement_data.csv: Cleaned dataset
- bid_predictions.json: Predicted price ranges
- dashboard_data.json: Visualization data
- procurement_dashboard.html: Interactive dashboard with research findings
